In [1]:
import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = 'N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO' 
CLIENT_SECRET = 'EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO
CLIENT_SECRET:EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

Data downloaded!


In [4]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Get New York Data

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [42]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Separate data by borough

In [8]:
boroughs=neighborhoods['Borough'].unique().tolist()
boroughs

['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island']

In [9]:
Brooklyn = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

for address in boroughs:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))
    
    LIMIT = 200 
    radius = 1000 
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
    print(url)

The geograpical coordinate of Bronx are 40.85048545, -73.8404035580209.
https://api.foursquare.com/v2/venues/explore?&client_id=N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO&client_secret=EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII&v=20180604&ll=40.85048545,-73.8404035580209&radius=1000&limit=200
The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.
https://api.foursquare.com/v2/venues/explore?&client_id=N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO&client_secret=EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII&v=20180604&ll=40.7900869,-73.9598295&radius=1000&limit=200
The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.
https://api.foursquare.com/v2/venues/explore?&client_id=N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO&client_secret=EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII&v=20180604&ll=40.6501038,-73.9495823&radius=1000&limit=200
The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.
https://api.foursquare.com/v2/v

### Top 200 venues in Brooklyn

In [10]:
results = requests.get('https://api.foursquare.com/v2/venues/explore?&client_id=N5TJZJ4ZQDQYX3YTGNLX5PHX1JK1JH550QPM4OMXKBBYRZHO&client_secret=EI02X1U00CFKYGU3WEQHTLXBTDWM3WAKAPEQF5T1NFBPEBII&v=20180604&ll=40.6501038,-73.9495823&radius=1000&limit=200').json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Exquisite Delight,Caribbean Restaurant,40.649091,-73.949243
1,Jus Juice,Juice Bar,40.652208,-73.949723
2,Nostrand Health Foods,Juice Bar,40.649092,-73.949243
3,"Courts (Furniture, Electronics, & Appliances)",Furniture / Home Store,40.650529,-73.950617
4,Tastee Pattee,Caribbean Restaurant,40.650850,-73.947838


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
brooklyn_venues = getNearbyVenues(names=Brooklyn['Neighborhood'],
                                   latitudes=Brooklyn['Latitude'],
                                   longitudes=Brooklyn['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [15]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,51,51,51,51,51,51
Bay Ridge,86,86,86,86,86,86
Bedford Stuyvesant,27,27,27,27,27,27
Bensonhurst,28,28,28,28,28,28
Bergen Beach,7,7,7,7,7,7
Boerum Hill,88,88,88,88,88,88
Borough Park,26,26,26,26,26,26
Brighton Beach,44,44,44,44,44,44
Broadway Junction,13,13,13,13,13,13


#### Analyse each neighborhood

In [15]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

brooklyn_onehot.drop('Neighborhood',axis=1)

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 
col_name = brooklyn_onehot.columns.tolist()
col_name.insert(0,'Neighborhood')
brooklyn_onehot.reindex(columns=col_name)

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()


,Yoga Studio,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wat

In [16]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()

#### Top 10 venues in Brooklyn

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Women's Store,Fast Food Restaurant,Sushi Restaurant,Pizza Place,Deli / Bodega,Donut Shop,Dessert Shop,Bubble Tea Shop
1,Bay Ridge,Italian Restaurant,Spa,Chinese Restaurant,Pizza Place,Thai Restaurant,Greek Restaurant,Bar,American Restaurant,Bagel Shop,Middle Eastern Restaurant
2,Bedford Stuyvesant,Deli / Bodega,Café,Pizza Place,Coffee Shop,Bus Stop,Bar,BBQ Joint,Fried Chicken Joint,Boutique,New American Restaurant
3,Bensonhurst,Ice Cream Shop,Donut Shop,Spa,Chinese Restaurant,Sushi Restaurant,Hotpot Restaurant,Cosmetics Shop,Café,Shabu-Shabu Restaurant,Smoke Shop
4,Bergen Beach,Harbor / Marina,Playground,Baseball Field,Donut Shop,Park,Athletics & Sports,Flower Shop,Field,Filipino Restaurant,Fish & Chips Shop


In [57]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels',axis=1,inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = Brooklyn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Italian Restaurant,Spa,Chinese Restaurant,Pizza Place,Thai Restaurant,Greek Restaurant,Bar,American Restaurant,Bagel Shop,Middle Eastern Restaurant
1,Brooklyn,Bensonhurst,40.611009,-73.995180,2,Ice Cream Shop,Donut Shop,Spa,Chinese Restaurant,Sushi Restaurant,Hotpot Restaurant,Cosmetics Shop,Café,Shabu-Shabu Restaurant,Smoke Shop
2,Brooklyn,Sunset Park,40.645103,-74.010316,2,Bank,Bakery,Pizza Place,Latin American Restaurant,Mexican Restaurant,Mobile Phone Shop,Pharmacy,Deli / Bodega,Gym,Supplement Shop
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Yoga Studio,Record Shop,Boutique,French Restaurant,Bakery,Café
4,Brooklyn,Gravesend,40.595260,-73.973471,2,Pizza Place,Bus Station,Lounge,Bakery,Baseball Field,Diner,Donut Shop,Pharmacy,Eastern European Restaurant,Electronics Store


#### Display cluster in Brooklyn

In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Find out frequency of 'Asian Restaurant' in top 10 most common venue

In [98]:
cluster0=brooklyn_merged.loc[brooklyn_merged['Cluster Labels']
                     == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

df0=cluster0.melt()
frequency0=len(df0[df0['value']=='Asian Restaurant'])/(len(cluster0.index)*10)
frequency0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Sea Gate,Home Service,Bus Station,Optical Shop,Beach,Spa,Food Truck,Food Stand,Food Court,Food & Drink Shop,French Restaurant
68,Madison,Bagel Shop,Spa,Dessert Shop,Pizza Place,Italian Restaurant,Deli / Bodega,Pilates Studio,Bus Station,Restaurant,Candy Store


In [184]:
cluster1=brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
df1=cluster1.melt()
frequency1=len(df1[df1['value']=='Asian Restaurant'])/(len(cluster1.index)*10)
frequency1
cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Canarsie,Asian Restaurant,Bus Line,Gym,Grocery Store,Chinese Restaurant,Event Service,Caribbean Restaurant,Women's Store,Fish Market,Flea Market
59,Paerdegat Basin,Food,Asian Restaurant,Chinese Restaurant,Bus Line,Gym,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Women's Store


In [100]:
cluster2=brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
df2=cluster2.melt()
frequency2=len(df2[df2['value']=='Asian Restaurant'])/(len(cluster2.index)*10)
frequency2

0.0

In [101]:
cluster3=brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
df3=cluster3.melt()
frequency3=len(df3[df3['value']=='Asian Restaurant'])/(len(cluster3.index)*10)
frequency3

0.0

In [102]:
cluster4=brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
df4=cluster4.melt()
frequency4=len(df4[df4['value']=='Asian Restaurant'])/(len(cluster4.index)*10)
frequency4

0.02

### Find out price range of 'Asian Restaurant' 

In [154]:
asian_cat=requests.get('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)).json()


In [20]:
latitude=40.6501038
longitude=-73.9495823
radius=1000
LIMIT=100
asian_url='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d142941735&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)

results_asian = requests.get(asian_url).json()
results_asian

{'meta': {'code': 200, 'requestId': '5d33b9372b274a002cfa2d88'},
 'response': {'venues': [{'id': '4d018c1c25b13704bd53c276',
    'name': 'New Chung Me',
    'location': {'address': '3017 Church Ave',
     'lat': 40.65083,
     'lng': -73.94874,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.65083,
       'lng': -73.94874}],
     'distance': 107,
     'postalCode': '11226',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['3017 Church Ave',
      'Brooklyn, NY 11226',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d145941735',
      'name': 'Chinese Restaurant',
      'pluralName': 'Chinese Restaurants',
      'shortName': 'Chinese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1563670839',
    'hasPerk': False},
   {'id': '4c8e5b0cd2aea093ebbbcd69',
    'name': 'Bai Li',
    'l

#### Get Asian Restaurant venue id

In [21]:
venue_ids=[]
for i in range(45):
    venue_id = results_asian['response']['venues'][i]['id']
    venue_ids.append(venue_id)

In [156]:
asian_lats=[]
asian_lngs=[]
for i in range(45):
    asian_lat = results_asian['response']['venues'][i]['location']['lat']
    asian_lats.append(asian_lat)
    asian_lng = results_asian['response']['venues'][i]['location']['lng']
    asian_lngs.append(asian_lng)

#### Find out how many restaurant in each price level

In [30]:
urls=[]
for venue in venue_ids:
    url1='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        venue,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT)
    urls.append(url1)

prices=[]
for url2 in urls:
    result_url = requests.get(url2).json()
    price=result_url['response']['venue']['price']['tier']
    prices.append(price)
    print(prices)
    print(result_url)
    

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0a2b274a002c636a26'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0abe7078003947068b'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0ad29cbb00237ed171'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0adeb4950032ddc49f'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0a9df1dc0023607f75'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0a429bfc00323ff39d'}, 'response': {}}
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5d33cd0b6adbf5003985a47e'}, '